In [2]:
import numpy as np 
import pandas as pd
import re
from IPython.display import display, HTML
import sklearn

## Read in the data: 

Data has been formatted for neural nets:
- Binary columns are encoded as -1/1 instead of as 0/1. 
- All columns are then scaled. 
- Categorical columns are one-hot encoded. 
- A few outliers removed. 

In [3]:
#note: these files do have headers 

train1 = pd.read_csv("lara_1_train.csv")    #The training data
test1 = pd.read_csv("lara_1_test.csv")      #Holdout data for testing on Kaggle

ytrain1 = pd.read_csv("y_1_train.csv")      #Training labels 

In [4]:
#convert from pandas to numpy arrays:
train1 = train1.as_matrix()
test1 = test1.as_matrix()
ytrain1 = np.array(ytrain1)[:,0]

In [5]:
print(train1.shape)
print(test1.shape)
print(ytrain1.shape)

(49352, 74)
(74659, 74)
(49352,)


For train/test splitting: 

In [ ]:
#from sklearn.model_selection import train_test_split 
#X_train, X_test, y_train, y_test = train_test_split(train1, ytrain1)

## Model 1:

log loss I got from this model on Kaggle: 0.62556

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss

In [14]:
model1 = MLPClassifier(activation = 'tanh', 
                    alpha =  1.0000000000000001e-05,
                    hidden_layer_sizes = (4,) ,
                    solver = 'sgd' )

In [27]:
model1.fit(train1, ytrain1)
y_pred_mod1 = model1.predict(test1)
y_pred_proba_mod1 = model1.predict_proba(test1)

For finding the log loss on a train/test split: 

In [ ]:
#print("log loss on the test set :")
#log_loss(y_test, y_pred_proba_mod1)  

## Model 2: 

In [ ]:
#!pip install tensorflow
#!pip install keras

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras import callbacks
from keras.layers.normalization import BatchNormalization
from keras import optimizers, initializers
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler

numpy.random.seed(7)

Using TensorFlow backend.


In [7]:
#Keras needs the labels to be in binary 0/1 vectors for each class: 
#categorical_labels_train = to_categorical(ytrain1-1, num_classes=None)

In [8]:
#scale = StandardScaler()
#train1 = scale.fit_transform(train1)
#test1 = scale.fit_transform(test1)

Import data with the above 2 steps completed:

In [22]:
train1 =  pd.read_csv("train1_allscaled.csv")    #The training data
categorical_labels_train = pd.read_csv("categorical_labels_train.csv")      #Training labels 

test1 = pd.read_csv("test1_allscaled.csv")

In [23]:
#convert from pandas to numpy arrays:
train1 = train1.as_matrix()
test1 = test1.as_matrix()
categorical_labels_train = np.array(categorical_labels_train )

In [24]:
print(train1.shape)
print(test1.shape)
print(categorical_labels_train.shape)

(49352, 74)
(74659, 74)
(49352, 3)


In [ ]:
model4 = Sequential()

inshape = train1.shape[1]

#Input layer: 
model4.add(Dense(50, input_dim= inshape, activation = 'tanh',         
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
                     
model4.add(BatchNormalization())
model4.add(Dropout(0.5))     

#One hidden layer: 
model4.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
model4.add(BatchNormalization())
model4.add(Dropout(0.25))

#Second hidden layer: 
model4.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
model4.add(BatchNormalization())
model4.add(Dropout(0.25))

#Output layer 
model4.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

#Setting up to optimize the weights: 
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#change verbose to 0 to stop printing the progress: 
model4.fit(train1,categorical_labels_train, epochs=150, batch_size=10, verbose=1)
    

Epoch 1/150
22710/49352 [============>.................] - ETA: 10s - loss: 0.7966 - acc: 0.6573

mean cross-validated testing log loss from this model :
0.612296414625 +/- 0.00209871497037